In [1]:
#Figure out if there were 2 or 3 hackers 
#Hackers split their work evenly, so:
    # #ofAttacks = 
    
#1. Start Spark Session
#2. IMport Data
#3. Explore Data
#4. Create 2 KMeans models (k=2,k=3)
#

In [5]:
import findspark
findspark.init('/home/ubuntu/spark-2.2.0-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kProj').getOrCreate()

# Import DAta 

In [6]:
folder = '/home/ubuntu/data/raw'
file = '/hack_data.csv'
data = spark.read.csv(folder+file, header=True, inferSchema=True)
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [7]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [8]:
import pandas as pd
import numpy as np
#Feature Correlations

from pyspark.sql.functions import corr, covar_pop
important_columns = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']


corr_df = pd.DataFrame(columns=important_columns, index=important_columns)
for feature in important_columns:
    for feature2 in important_columns:
        #new_data.select(corr(feature,feature2)).show()
        corr_df[feature2][feature] = data.select(corr(feature,feature2)).head()[0]
corr_df.head()


,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
Session_Connection_Time,1,0.713861,0.0346872,0.808394,0.844167,None,-0.866077
Bytes Transferred,0.713861,1,0.0724358,0.739822,0.784081,None,-0.793344
Kali_Trace_Used,0.0346872,0.0724358,1,0.0332419,0.0169312,None,-0.0265603
Servers_Corrupted,0.808394,0.739822,0.0332419,1,0.89721,None,-0.915629
Pages_Corrupted,0.844167,0.784081,0.0169312,0.89721,1,None,-0.968662


In [13]:
#Should we transform the Location column?
data.select('Location').show(5)

#Naw, all different

+--------------------+
|            Location|
+--------------------+
|            Slovenia|
|British Virgin Is...|
|             Tokelau|
|             Bolivia|
|                Iraq|
+--------------------+
only showing top 5 rows



# Transform Features

In [17]:
from pyspark.ml.feature import VectorAssembler
features = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
# 'Location',
 'WPM_Typing_Speed']
assembler = VectorAssembler(inputCols=features, outputCol='features')
final_data = assembler.transform(data)
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [24]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features',
                        outputCol='scaledFeatures')
scaler_fitted = scaler.fit(final_data)
final_data = scaler_fitted.transform(final_data)
final_data.printSchema()


root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



# Create and Test Clusters

In [28]:
from pyspark.ml.clustering import KMeans
k_clf2 = KMeans(k=2, featuresCol='scaledFeatures')
k_clf3 = KMeans(k=3, featuresCol='scaledFeatures')
k_clf4 = KMeans(k=4, featuresCol='scaledFeatures')
k_fitted2 = k_clf2.fit(final_data)
k_fitted3 = k_clf3.fit(final_data)
k_fitted4 = k_clf4.fit(final_data)


In [29]:
print('WSSE - 2 Clusters', k_fitted2.computeCost(final_data))
print('WSSE - 3 Clusters', k_fitted3.computeCost(final_data))
print('WSSE - 4 Clusters', k_fitted4.computeCost(final_data))

WSSE - 2 Clusters 601.7707512676716
WSSE - 3 Clusters 434.1492898715845
WSSE - 4 Clusters 267.1336116887891


In [127]:
k_fitted2.transform(final_data).select('prediction').count()
k_fitted2.transform(final_data).select('prediction').head(334)[332][0]
k_fitted2.transform(final_data).groupBy('prediction').count().show()
k_fitted3.transform(final_data).groupBy('prediction').count().show()
k_fitted4.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         3|   79|
|         2|   83|
|         0|   88|
+----------+-----+



In [119]:
count1
#k_fitted2.transform(final_data).select('prediction').head().count()

0

In [118]:
#Check how many attacks belong to each cluster
i = 0
count0 = 0
count1 = 0
for row in final_data:
    prediction = k_fitted2.transform(final_data).select('prediction').head(10000)[i][0]
    if (int(prediction) == 1):
        count0 = count0 + 1
    elif (int(prediction) == 0):
        count1 = count1 + 2
    i += 1